**Name:** Rafael Henrique Martello

**Date**: 05-09-2021

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 2 - Hypothesis Testing


Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    # Load Dataset and set Column 0 name to State
    df_ut = pd.read_csv('Data_A2/university_towns.txt',sep='/n',engine='python',header=None)
    df_ut = df_ut.rename(columns={0:'State'})

    # get RegionName where state cotain [edit]
    df_ut.loc[~df_ut['State'].str.contains("[edit]", regex=False), 'RegionName'] = df_ut['State']

    df_ut.loc[df_ut['State']==df_ut['RegionName'],'State'] = np.nan

    # remove [ from State and fillna with method ffil (propagate last valid observation forward to next)
    df_ut['State'] = (df_ut['State'].str.replace(r'\[.*','', regex=True).fillna(method='ffill'))
    
    # remove ( from RegionName
    df_ut['RegionName'] = df_ut['RegionName'].str.replace(r'\W*\(.*','', regex=True)

    df_ut = df_ut.drop(df_ut[df_ut['RegionName'].isna()].index)
    
    return df_ut

    #state_college_df = pd.DataFrame(state_town,columns = ['State','RegionName'])
    #return state_college_df
        

get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
...,...,...
561,Wisconsin,River Falls
562,Wisconsin,Stevens Point
563,Wisconsin,Waukesha
564,Wisconsin,Whitewater


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('Data_A2/gdplev.xls', skiprows=7) 

    # get columns referring to Quarter and GDP for Quarter
    gdp2 = gdp[['Unnamed: 4', 'Unnamed: 5']]
    
    # set column names
    gdp2.columns = ['Quarter','GDP']
    # select data starting in the year 2000
    gdp2 = gdp2[gdp2['Quarter']>'2000'].copy()
    

    #     # if the GDP from i is higher that the i+1 and the GDP of i+i is higher that i+2 (two consecutively lower values)
    start_recession = gdp2[(gdp2['GDP'] > gdp2['GDP'].shift(-1))&(gdp2['GDP'].shift(-1) > gdp2['GDP'].shift(-2))].copy()
    
    return start_recession.iloc[0,0]
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''


    gdp = pd.read_excel('Data_A2/gdplev.xls', skiprows=7)
    
    # get columns referring to Quarter and GDP for Quarter
    gdp2 = gdp[['Unnamed: 4', 'Unnamed: 5']]
    
    # set column names
    gdp2.columns = ['Quarter','GDP']

    # get the star of recession
    start_recession = get_recession_start()

    # appy the star of recession to the dataset
    gdp2 = gdp2[gdp2['Quarter'] >= start_recession].copy()
    
    end_recession = gdp2[(gdp2['GDP'] > gdp2['GDP'].shift(1))&(gdp2['GDP'].shift(1) > gdp2['GDP'].shift(2))].copy()
    
    return end_recession.iloc[0,0]

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('Data_A2/gdplev.xls', skiprows=7)
    
    # get columns referring to Quarter and GDP for Quarter
    gdp2 = gdp[['Unnamed: 4', 'Unnamed: 5']]
    
    # set column names
    gdp2.columns = ['Quarter','GDP']

    # get star and end of the recession
    start_recession = get_recession_start()
    end_recession = get_recession_end()

    # filter dataset to get only the data in between start and end
    gdp2 = gdp2[(gdp2['Quarter'] >= start_recession)&(gdp2['Quarter'] <= end_recession)].copy()
    
    # calculate de bottom of the recession
    bottom_recession = gdp2[gdp2['GDP']==gdp2['GDP'].min()].copy()
    
    return bottom_recession.iloc[0,0]

get_recession_bottom()    

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # First defining the quarters of each month through a dictionary
    month_to_qtr = {'01':'q1','02':'q1','03':'q1','04':'q2','05':'q2','06':'q2',
                '07':'q3','08':'q3','09':'q3','10':'q4','11':'q4','12':'q4'}

    df_house = pd.read_csv('Data_A2/City_Zhvi_AllHomes.csv',header = 0)

    # set columns to drop
    columns_d = list(range(3,51))
    
    # drop columns from 3 to 50
    df_house.drop(df_house.columns[columns_d],axis=1,inplace=True)
    
    # drop column 0
    df_house.drop(df_house.columns[0],axis=1,inplace=True)
    
    # replaces the initials of the states with the name according to the dictionary
    df_house.replace({'State':states}, inplace = True)
    
    # set index as State and region
    df_house.set_index(['State','RegionName'],inplace = True)

    # groupby data and create a fixed-frequency using PeriodIndex in quarters for the columns 
    df_house = df_house.groupby(pd.PeriodIndex(df_house.columns, freq='Q'), axis=1).mean()
    
    # set dataframe
    df_house = pd.DataFrame(df_house)
    
    return df_house


convert_housing_data_to_quarters()

,,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    ## Load data from other defs
    start_recession = get_recession_start()
    bottom_recession = get_recession_bottom()
    towns = get_list_of_university_towns()

    # create column type to identify university towns
    towns['Type'] = 'Uni'
    
    # select data from convert_housing_data_to_quarters in the intervallstart_recession to bottom_recession
    df_house = convert_housing_data_to_quarters()[[start_recession,bottom_recession]].copy()

    # calculate price_ratio
    df_house['Ratio'] = df_house[start_recession]/df_house[bottom_recession]

    # copy the price ratio that is not NA
    df_house = df_house[df_house['Ratio'].notna()].copy()
    
    # join left table towns and df_house
    df_house = pd.merge(df_house, towns, on=['State','RegionName'], how='left')

    # set empty type to Non
    df_house['Type'] = df_house['Type'].fillna('Non')
    
    # divide in two tables uni and non uni
    uni = df_house[df_house['Type']=='Uni'].copy()
    non = df_house[df_house['Type']=='Non'].copy()
    
    # calcule test_p using scipy.stats.ttest_ind() and verify if is different or not
    p = ttest_ind(uni['Ratio'], non['Ratio'])[1]
    if p < 0.01:
        different = True
    else:
        different = False
    
    # verify if university town is better or not 
    if uni['Ratio'].mean() < non['Ratio'].mean():
        better = "university town"
    else:
        better = "non-university town"
    
    return (different, p, better)

run_ttest()

(True, 0.0054964273536938875, 'university town')